 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
import pandas as pd
import synonyms
import jieba
import jieba.posseg as pseg
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

Model loaded succeed
>> Synonyms on loading ...


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [2]:
corpus_df = pd.read_csv('./corpus.csv').reset_index(drop = True)
corpus_2_list = corpus_df[corpus_df['rate'] == 2]['token'].tolist()
corpus_3_list = corpus_df[corpus_df['rate'] == 3]['token'].tolist()

In [4]:
def rate_token(token, corpus_3, corpus_2):
    '''
    Input: One token
    Output: Rate 
    '''
    if token in corpus_3:
        return 3
    elif token in corpus_2:
        return 2
    else:
        return 0

In [7]:
def rate_title_tokens(tokens):
    token_rates = list(map(token_rater, tokens))
    return token_rates

In [5]:
def rate_title_syns(tokens):
    
    syn_3_cnt = 0
    syn_2_cnt = 0
    syn_cnt_list = []
    
    for token in tokens:
        syn = synonyms.nearby(token)[0]
        syn_rates = list(map(token_rater, syn))

        syn_cnt = [2 if syn_rates.count(3) < syn_rates.count(2) else(0 if syn_rates.count(3) == 0 and syn_rates.count(2) == 0 else 3)]
        syn_cnt_list += syn_cnt
        
    return syn_cnt_list

In [6]:
def title_processor(title):
    
    tokens = list(set(list(jieba.cut_for_search(title))))

    token_cnt_list = np.array(title_token_rater(tokens))
    syn_cnt_list = np.array(title_syn_rater(tokens))
    
    token_non_zero = np.where(token_cnt_list > 0)
    syn_cnt_list = np.delete(syn_cnt_list, token_non_zero)
            
    token_cnt_list = token_cnt_list.tolist()
    syn_cnt_list = syn_cnt_list.tolist()
    
    syn_3_cnt = syn_cnt_list.count(3)
    syn_2_cnt = syn_cnt_list.count(2)
    token_3_cnt = token_cnt_list.count(3)
    token_2_cnt = token_cnt_list.count(2)
    
    return [token_2_cnt, token_3_cnt, syn_2_cnt, syn_3_cnt]

In [14]:
# miaozhen sample dataset

training_title_list = open('/Users/olap/Desktop/training_title_list.txt', 'r').read().split('\n')[:100]
title_feature_list = list(map(title_processor, training_title_list))

ture_label = open('/Users/olap/Desktop/training_set.txt', 'r').read().split('\n')
true_label_list = list(map(lambda x:x.split(' ')[-1], ture_label))
true_label_list = [1 if label=='0' else label for label in true_label_list]
true_label_list = [0 if label=='100' else label for label in true_label_list][:-1]

title = pd.DataFrame(training_title_list)
feature = pd.DataFrame(title_feature_list)
true_label = pd.DataFrame(true_label_list)

noobs_df = pd.concat([title, feature, true_label], axis= 1)
noobs_df.columns = ['title', 'token_2', 'token_3', 'syn_2', 'syn_3', 'true_label']

In [16]:
# yewai title

yewai_df = pd.read_csv('/Users/olap/Desktop/yw_title.csv').drop('Unnamed: 0', axis  = 1)

yewai_df = yewai_df[yewai_df.true_label.notnull()].reset_index(drop = True)

In [17]:
rf = RandomForestClassifier()
lr = linear_model.LogisticRegression()

In [18]:
def model_perfermance(data, test_size, model, diff):
    
    title_list = data['title'].tolist()
    X = np.array(list(map(title_processor, title_list)))
    y = np.array(data['true_label'].tolist())
    
    title = pd.DataFrame(title_list)
    feature = pd.DataFrame(X)
    true_label = pd.DataFrame(y)

    df = pd.concat([title, feature, true_label], axis= 1)
    df.columns = ['title', 'token_2', 'token_3', 'syn_2', 'syn_3', 'true_label']
    
    # split data
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=0)
    
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    #train model
    model.fit(X_train, y_train)
    
    print('traning acc', model.score(X_train, y_train))
    print('test acc', model.score(X_test, y_test))

    # predicted probalibility
    model_predict = pd.DataFrame(model.predict(X_test))
    model_predict.columns = ['pred_label']
    model_proba = pd.DataFrame(model.predict_proba(X_test))
    model_proba.columns = ['proba_0', 'proba_1']
    
    #get test set
    index = test_index.tolist()
    test_df = df.iloc[index].reset_index(drop = True)
    test_df = pd.concat([test_df, model_predict, model_proba], axis= 1)
    
    # rule base perfermance    
    rule_base_mis = len(test_df[((test_df['token_3'] == 0) & (test_df['true_label'] == 1))
             | (test_df['token_3'] > 0) & (test_df['true_label'] == 0)])
    
    print('rule base acc', 1-rule_base_mis/len(test_df))
    
    # mistake cases
    mis_df = test_df[test_df['true_label'] != test_df['pred_label']]
    
    # title rated manually
    manual_rating_df = test_df[abs(test_df['proba_0']-test_df['proba_1']) < diff] 
    manual_rating_cnt = len(manual_rating_df)
    manual_improve_cnt = len(test_df[(test_df['true_label'] != test_df['pred_label'])
                            & (abs(test_df['proba_0']-test_df['proba_1']) < diff)])
    
    print('test_count', len(test_df))
    print('manually rate title', manual_rating_cnt)
    print('can improve', manual_improve_cnt)
    
    if manual_rating_cnt != 0:
        print('manyally eff', manual_improve_cnt/manual_rating_cnt)
    
    print('model + manual acc', model.score(X_test, y_test) + manual_improve_cnt/len(test_df))
    
    #return mistake cases to imporve model/corpus
    return mis_df

In [22]:
model_perfermance(noobs_df, .4, lr, .3)

traning acc 0.9
test acc 0.95
rule base acc 0.875
test_count 40
manually rate title 10
can improve 2
manyally eff 0.2
model + manual acc 1.0


,title,token_2,token_3,syn_2,syn_3,true_label,pred_label,proba_0,proba_1
2,分手你还赚了一个处，你不亏！,1,0,0,0,1,0,0.544775,0.455225
7,"舆情分析：兰陵一副校长致初中女生""怀孕"" - 案例库 - 华声舆情 - 华声在线",1,0,1,0,1,0,0.506143,0.493857
